In [6]:
import copy
import math
import keras 
import rawpy
import random

import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from PIL import Image
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM,Embedding,Dropout,Reshape
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.python.keras import backend as K
from tensorflow.python.client import device_lib
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn

import math
import matplotlib.pyplot as plt
torch.manual_seed(11)
import pyspark 
from pyspark.sql import SparkSession

In [12]:

train_path = "./Training"
train_path_fire = train_path+ "/Fire"
train_path_no_fire = train_path+ "/No_Fire"

In [13]:
size = (254,254,3)

In [15]:
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Protob Conversion to Parquet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.format("image").option("dropInvalid", True).load(train_path)
df.select("image.origin", "image.width", "image.height").show(truncate=False)



Py4JJavaError: An error occurred while calling o79.load.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1215)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1420)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(Trave

In [ ]:
prev_model = tf.keras.applications.EfficientNetV2B1(
    weights=None,
    input_tensor=None,
    input_shape=(45,64,4),
    pooling=None,
    include_preprocessing=True,
)


prev_model = tf.keras.applications.MobileNet(
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(45,64,4),
    pooling=None
)

model = Sequential()
model.add(prev_model)
#prev_model.summary()
model.add(Reshape((4,250,)))
#model.add(Reshape((1,1000,)))
#model.add(Flatten)
model.add(LSTM(256, activation='tanh',input_dim=4, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, activation='tanh', return_sequences=True))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Flatten()) # tf.keras.layers.Flatten() 
model.add(Dense(5760, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64*45, activation='sigmoid'))
